In [24]:
#This code was execture on google cloud as it was too bulky for my laptop

import numpy as np 
import pandas as pd 
import librosa 
import os 
import soundfile as sf
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [25]:
def initialise_training_set():
    path_female = r'C:\Users\Atulya\Documents\GitHub\gender-classifier-using-voice\Data Preprocessing\Female'
    path_male = r'C:\Users\Atulya\Documents\GitHub\gender-classifier-using-voice\Data Preprocessing\Male'
    m=[]
    f=[] 
    for audiofile in os.listdir(path_male):
        try:
            y, sr = sf.read(os.path.join(path_male,audiofile))
            y = librosa.resample(y, sr, 8000)
            y = y[0:40000]
            y = np.concatenate((y, [0]* (40000 - y.shape[0])), axis=0)
            m.append(y)
        except RuntimeError:
            print(".DS_Store file detected and dismissed")
            pass
    for audiofile in os.listdir(path_female):     
        try:
            y, sr = sf.read(os.path.join(path_female,audiofile))
            y = librosa.resample(y, sr, 8000)
            y = y[0:40000]
            y = np.concatenate((y, [0]* (40000 - y.shape[0])), axis=0)
            f.append(y)
        except RuntimeError:
            print(".DS_Store file detected and dismissed")   
            pass
    labels_m = [1] * 3858
    labels_f = [0] * 3858
    set_m = list(zip(m,labels_m))
    set_f = list(zip(f,labels_f))
    data = set_m + set_f 
    df = pd.DataFrame(data, columns = ['Audio Data', 'Label'])
    return df

In [26]:
def initialise_test_set():
    m=[]
    f=[]
    path_female = r'C:\Users\Atulya\Documents\GitHub\gender-classifier-using-voice\Data Preprocessing\F6'
    path_male = r'C:\Users\Atulya\Documents\GitHub\gender-classifier-using-voice\Data Preprocessing\Male\M6'
    for audiofile in os.listdir(path_male):
        try:
            y, sr = sf.read(os.path.join(path_male,audiofile))
            y = librosa.resample(y, sr, 8000)
            y = y[0:40000]
            y = np.concatenate((y, [0]* (40000 - y.shape[0])), axis=0)
            m.append(y)
        except RuntimeError:
            print(".DS_Store file detected and dismissed")
            pass
    for audiofile in os.listdir(path_female):     
        try:
            y, sr = sf.read(os.path.join(path_female,audiofile))
            y = librosa.resample(y, sr, 8000)
            y = y[0:40000]
            y = np.concatenate((y, [0]* (40000 - y.shape[0])), axis=0)
            f.append(y)
        except RuntimeError:
            print(".DS_Store file detected and dismissed")   
            pass
    labels_m = [1] * 3858
    labels_f = [0] * 3858
    set_m = list(zip(m,labels_m))
    set_f = list(zip(f,labels_f))
    data = set_m + set_f 
    df = pd.DataFrame(data, columns = ['Audio Data', 'Label'])
    return df

In [27]:
def shuffle_training_set(train):
    train = train.reindex(np.random.permutation(train.index))
    return train

In [28]:
def shuffle_test_set(test):
    test = test.reindex(np.random.permutation(test.index))
    return test

In [29]:
def from_dataframe_to_array(train_shuff,test_shuff):
    train = train_shuff.values 
    test = test_shuff.values 
    x_train = np.zeros(shape=(train.shape[0],40000))
    x_test = np.zeros(shape=(test.shape[0],40000))
    for i in range(0,train.shape[0]):
        for j in range(0,40000):
            x_train[i][j] = train[i][0][j]
    x_test = np.zeros(shape=(test.shape[0],40000))
    for i in range(0,test.shape[0]):
        for j in range(0,40000):
            x_test[i][j] = test[i][0][j]        
    y_train = train_shuff["Label"][:]
    y_test = test_shuff["Label"][:]
    return x_train, x_test, y_train, y_test 

In [30]:
def model(x_train,x_test,y_train,y_test):
    x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)
    model = Sequential() 
    model.add(LSTM(512, input_shape=(x_train.shape[1:]), kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(128, kernel_initializer='normal', activation = 'relu'))
    model.add(Dropout(0.30))
    model.add(Dense(16, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal', activation = 'sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy','binary_accuracy',])
    model.fit(x_train, y_train, epochs=20, batch_size=2048)
    score = model.evaluate(x_test, y_test, batch_size=100)

In [31]:
def main():
    train = initialise_training_set()
    test = initialise_test_set()
    train_shuff = shuffle_training_set(train)
    test_shuff = shuffle_test_set(test)
    x_train, x_test, y_train, y_test = from_dataframe_to_array(train_shuff,test_shuff)
    model(x_train,x_test,y_train,y_test)

In [ ]:
main()

Epoch 1/20
